In [2]:
import pandas as pd 
import numpy as np 
from pandas.api.types import is_numeric_dtype

In [3]:
a = pd.DataFrame(np.array([['Cold', 105.6, 271.47, 48.0, 88.1], 
                                      ['hot', 77.75, 149.52, 80.7, 30.0],
                                      ['hot', 15.16, 47.41, 140.2, 25.0],
                                      ['hot', 14.63, 41.72, 59.8, 25.0],
                                      ['hot', 5.25, 34.79, 80.0, 25.0]]), 
                            columns = ['stream_type', 'mass_flow_rate', 
                                       'flowing_heat_capacity', 'initial_temp', 'final_temp'])

a

,stream_type,mass_flow_rate,flowing_heat_capacity,initial_temp,final_temp
0,Cold,105.6,271.47,48.0,88.1
1,hot,77.75,149.52,80.7,30.0
2,hot,15.16,47.41,140.2,25.0
3,hot,14.63,41.72,59.8,25.0
4,hot,5.25,34.79,80.0,25.0


In [4]:
a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   stream_type            5 non-null      object
 1   mass_flow_rate         5 non-null      object
 2   flowing_heat_capacity  5 non-null      object
 3   initial_temp           5 non-null      object
 4   final_temp             5 non-null      object
dtypes: object(5)
memory usage: 328.0+ bytes


In [5]:
def dataframe_converter(df):
    df = df.transform(pd.to_numeric, errors = 'ignore')
    return df

In [6]:
a = dataframe_converter(a)

In [7]:
a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   stream_type            5 non-null      object 
 1   mass_flow_rate         5 non-null      float64
 2   flowing_heat_capacity  5 non-null      float64
 3   initial_temp           5 non-null      float64
 4   final_temp             5 non-null      float64
dtypes: float64(4), object(1)
memory usage: 328.0+ bytes


In [12]:
def generate_heat_load(df):
    df['heat_load'] = (df['final_temp'] - df['initial_temp']) * df['flowing_heat_capacity']
    df['heat_load'] = df['heat_load'].abs()
    return df

In [28]:
a = generate_heat_load(a)
a

,stream_type,mass_flow_rate,flowing_heat_capacity,initial_temp,final_temp,heat_load
0,Cold,105.60,271.47,48.0,88.1,10885.947
1,hot,77.75,149.52,80.7,30.0,7580.664
2,hot,15.16,47.41,140.2,25.0,5461.632
3,hot,14.63,41.72,59.8,25.0,1451.856
4,hot,5.25,34.79,80.0,25.0,1913.450


In [37]:
b = a[['stream_type', 'initial_temp', 'final_temp']].melt('stream_type', 
                                                      value_name = 'temp').sort_values('temp', ascending = False).reset_index(drop = True)

In [49]:
c = a[['initial_temp', 'final_temp', 'flowing_heat_capacity']].melt('flowing_heat_capacity', 
                                                                    value_name = 'temp').sort_values('temp', ascending = False).reset_index(drop = True)

c

,flowing_heat_capacity,variable,temp
0,47.41,initial_temp,140.2
1,271.47,final_temp,88.1
2,149.52,initial_temp,80.7
3,34.79,initial_temp,80.0
4,41.72,initial_temp,59.8
5,271.47,initial_temp,48.0
6,149.52,final_temp,30.0
7,47.41,final_temp,25.0
8,41.72,final_temp,25.0
9,34.79,final_temp,25.0


In [43]:
a[['stream_type', 'flowing_heat_capacity']]

,stream_type,flowing_heat_capacity
0,Cold,271.47
1,hot,149.52
2,hot,47.41
3,hot,41.72
4,hot,34.79


In [38]:
b

,stream_type,variable,temp
0,hot,initial_temp,140.2
1,Cold,final_temp,88.1
2,hot,initial_temp,80.7
3,hot,initial_temp,80.0
4,hot,initial_temp,59.8
5,Cold,initial_temp,48.0
6,hot,final_temp,30.0
7,hot,final_temp,25.0
8,hot,final_temp,25.0
9,hot,final_temp,25.0


In [55]:
c

,flowing_heat_capacity,variable,temp
0,47.41,initial_temp,140.2
1,271.47,final_temp,88.1
2,149.52,initial_temp,80.7
3,34.79,initial_temp,80.0
4,41.72,initial_temp,59.8
5,271.47,initial_temp,48.0
6,149.52,final_temp,30.0
7,47.41,final_temp,25.0
8,41.72,final_temp,25.0
9,34.79,final_temp,25.0


In [72]:
d = pd.merge(b,c, how = 'left', on =['variable', 'temp'])
d

,stream_type,variable,temp,flowing_heat_capacity
0,hot,initial_temp,140.2,47.41
1,Cold,final_temp,88.1,271.47
2,hot,initial_temp,80.7,149.52
3,hot,initial_temp,80.0,34.79
4,hot,initial_temp,59.8,41.72
5,Cold,initial_temp,48.0,271.47
6,hot,final_temp,30.0,149.52
7,hot,final_temp,25.0,47.41
8,hot,final_temp,25.0,41.72
9,hot,final_temp,25.0,34.79


In [15]:
def problem_table_algorithm(df):
    df = df[['stream_type', 'initial_temp', 'final_temp']].melt('stream_type', 
                                                      value_name = 'temp').sort_values('temp', 
                                                                                       ascending = False).reset_index(drop = True)
    

,stream_type,mass_flow_rate,flowing_heat_capacity,initial_temp,final_temp,heat_load
0,Cold,105.60,271.47,48.0,88.1,10885.947
1,hot,77.75,149.52,80.7,30.0,7580.664
2,hot,15.16,47.41,140.2,25.0,5461.632
3,hot,14.63,41.72,59.8,25.0,1451.856
4,hot,5.25,34.79,80.0,25.0,1913.450


In [7]:
def min_temp_diff_selection(t_min = 12, tradeoff = 'energy'):
    """
    This determines the max possible amount of heat recovery.
    Optimium delta_T_min is influenced by the tradeoff between capital and energy costs.
    
    default t_min set at 12 because increasing t_min decreases annualised capital cost. 
    However, this increases energy costs (so
    if your process has a lot of energy cost requirements, you'd want to decrease t_min)
    """
    if tradeoff == 'energy':
        6 <= t_min < 12
    
    
    elif tradeoff == 'capital':
        12 <= t_min <= 20

# ----------------------- USELESS FUNCTIONS -----------------------

In [ ]:
#def datatype_converter(df):
#    col_list = []
#    for column in df.columns:
#        col_list.append(column)
#        for i in range(len(col_list)):
#            if is_string_dtype(df[col_list.pop()]):
#                df.apply(pd.to_numeric, errors = 'coerce') # coerce invalid values to nan. 
#            else:
#                pass
#    return df

In [ ]:
def datatype_converter1(df):
    target_cols = [col for col in df.columns if is_numeric_dtype(df[col])]
    for col in target_cols:
        df.loc[:, col] = pd.to_numeric(df[col])
    return df

In [ ]:
a = datatype_converter1(a)

In [ ]:
a

In [ ]:
def datatype_converter2(df):
    ind = df.columns != 'stream_type'
    df.loc[:, ind] = df.loc[:, ind].astype(float)
    return df

In [ ]:
ind = a.columns != 'stream_type'
ind

In [ ]:
a = datatype_converter2(a)

In [ ]:
a.info()

In [ ]:
a = a.transform(pd.to_numeric, errors = 'ignore')

In [ ]:
a.info()